In [ ]:
import os
import re
import sys
import numpy as np
import pandas as pd
import xarray as xr
from natsort import natsorted
from scipy.stats import zscore
from os.path import join as pjoin
import plotly.graph_objects as go

sys.path.append('../../')
import mannkendall as mk
import plotting_functions as pf
import circletrack_neural as ctn

In [ ]:
## Settings
project_dir = 'MultiCon_Imaging'
experiment_dir = 'MultiCon_Imaging5'
output_path = os.path.abspath(f'../../../{project_dir}/{experiment_dir}/output')
mouse_list = ['mc44', 'mc46', 'mc49', 'mc51', 'mc52']
mouse = 'mc51'
alpha = 0.001 ## for Mann-Kendall trend test

In [ ]:
## Can use this code block to check individual sessions and QC by eye units that the Mann-Kendall trend test is detecting to be dropped
session_idx = 11
c_path = pjoin(output_path, f'aligned_minian/{mouse}/C')
yra_path = pjoin(output_path, f'aligned_minian/{mouse}/YrA')
s_path = pjoin(output_path, f'aligned_minian/{mouse}/S')
place_path = pjoin(output_path, f'aligned_place_cells/{mouse}/S')

c_save = pjoin(output_path, f'aligned_minian_two/{mouse}/C')
yra_save = pjoin(output_path, f'aligned_minian_two/{mouse}/YrA')
s_save = pjoin(output_path, f'aligned_minian_two/{mouse}/S')
place_save = pjoin(output_path, f'aligned_place_cells_two/{mouse}/S')
for idx, session in enumerate(natsorted(os.listdir(c_path))):
    if idx == session_idx:
        day = re.search('_([0-9]+)', session)[0]
        fig = pf.custom_graph_template(x_title='Time (s)', y_title='Activity (a.u.)', width=700)
        cell_trends = {'unit_id': [], 'trend': [], 'mk_score': [], 'pval': []}

        ## Load data
        C = xr.open_dataset(pjoin(c_path, session))['C']
        if len(day) == 2:
            yra_string = f'{mouse}_YrA_{day[-1]}.nc'
            s_string = f'{mouse}_S_{day[-1]}.nc'
            c_string = f'{mouse}_C_{day[-1]}.nc'
        else:
            yra_string = f'{mouse}_YrA_{day[-2:]}.nc'
            s_string = f'{mouse}_S_{day[-2:]}.nc'
            c_string = f'{mouse}_C_{day[-2:]}.nc'
        YrA = xr.open_dataset(pjoin(yra_path, yra_string))['YrA']
        S = xr.open_dataset(pjoin(s_path, s_string))['S']
        S_place = xr.open_dataset(pjoin(place_path, s_string))['S']

        ## Mann-Kendall trend test on taking the minimum value in a 20-second bin of z-scored data
        ## Rationale is that if there is an increasing/decreasing trend, the z-scored values get brought above or
        ## below zero, and Mann-Kendall can detect this. 20-second bin allows for less false positives
        cell_act = C.values
        zdata = zscore(cell_act, axis=1)
        bin_act = ctn.bin_activity(zdata, bin_size_seconds=20, func=np.min)
        for uid in np.arange(0, C.shape[0]):
            mkt = mk.original_test(bin_act[uid, :])
            cell_trends['unit_id'].append(uid)
            cell_trends['trend'].append(mkt.trend)
            cell_trends['mk_score'].append(np.abs(mkt.s))
            cell_trends['pval'].append(mkt.p)
        trends = pd.DataFrame(cell_trends)
        dropped_uids = trends[(trends['pval'] < alpha)].reset_index(drop=True)

        for neuron in dropped_uids['unit_id']:
            fig.add_trace(go.Scattergl(x=YrA['behav_t'].values, y=YrA.values[neuron, :], mode='lines', visible='legendonly',
                                    line_color='black', name=f'Neuron {neuron}', showlegend=False, legendgroup=f'{neuron}'))
            fig.add_trace(go.Scattergl(x=C['behav_t'].values, y=zdata[neuron, :], mode='lines', visible='legendonly',
                                    line_color='red', name=f'Neuron {neuron}', legendgroup=f'{neuron}'))
    else:
        pass
fig.show()

In [ ]:
## Alter boolean to keep cells you don't want to drop
## The Neuron number on the graph corresponds to the index
keep_cell_ids = [439]
to_be_dropped = (trends['pval'] < alpha).to_numpy()
if dropped_uids.shape[0] == 0:
    print('All units passed quality control!')
elif len(keep_cell_ids) == 0:
    print('Dropping all identified units!')
else:
    print('Keeping selected units...')
    for val in keep_cell_ids:
        to_be_dropped[val] = False

In [ ]:
## Assign boolean array as a coordinate to each activity array
C = C.assign_coords(passed_qc=('unit_id', ~to_be_dropped))
S = S.assign_coords(passed_qc=('unit_id', ~to_be_dropped))
YrA = YrA.assign_coords(passed_qc=('unit_id', ~to_be_dropped))
S_place = S_place.assign_coords(passed_qc=('unit_id', ~to_be_dropped))

## Save output
if not os.path.exists(c_save):
    os.makedirs(c_save)

if not os.path.exists(s_save):
    os.makedirs(s_save)

if not os.path.exists(yra_save):
    os.makedirs(yra_save)

if not os.path.exists(place_save):
    os.makedirs(place_save)
    
C.to_netcdf(pjoin(c_save, c_string))
S.to_netcdf(pjoin(s_save, s_string))
YrA.to_netcdf(pjoin(yra_save, yra_string))
S_place.to_netcdf(pjoin(place_save, s_string))

### Look for a threshold to reduce non-existent spikes in S to zero.

In [ ]:
neuron = 8
single_neuron_thresh = ctn.calculate_otsu_thresh(S[neuron, :])
all_neuron_thresh = ctn.calculate_otsu_thresh(S.values)
scale_factor = 10

fig = pf.custom_graph_template(x_title='Time (s)', y_title='Activity (a.u.)', width=1400, rows=1, columns=3,
                               shared_x=True, shared_y=True, titles=['Original', 'Single Neuron Otsu', 'All Neuron Otsu'])
for col_val in np.arange(1, 4):
    fig.add_trace(go.Scattergl(x=YrA['behav_t'].values, y=YrA.values[neuron, :], mode='lines', line_color='black', 
                            name=f'Neuron {neuron}', showlegend=False, legendgroup=f'{neuron}'), row=1, col=col_val)

    if col_val == 1:
        output = S.copy()
    elif col_val == 2:
        output = S.copy()
        output[neuron, output[neuron, :] <= (single_neuron_thresh / 2)] = 0
    elif col_val == 3:
        output = S.copy()
        output[neuron, output[neuron, :] <= all_neuron_thresh] = 0

    fig.add_trace(go.Scattergl(x=S['behav_t'].values, y=output[neuron, :] * scale_factor, mode='lines', line_color='red', 
                            name=f'Neuron {neuron}', legendgroup=f'{neuron}', showlegend=False), row=1, col=col_val)
fig.show()

### Look at other members in the lab's data.

In [ ]:
## Settings for Joe's mice
experiment = 'RLI6'
mouse = f'{experiment}-4'
session = 'OfflineDay1'
time = '16_31_57'
dpath = f'/media/caishuman/csstorage/Joe/Experiments/Retrospective_Linking/RL_Imaging/{experiment}/{experiment}_Experiment/{mouse}/{mouse}_{session}/{time}/Miniscope/minian'
alpha = 0.01

In [ ]:
## Settings for Chemotagged mice
mouse = 'GCaMP2'
session = 'PreSession1'
dpath = f'/media/caishuman/csstorage3/Destynie/Misc/GAD_hM3Dq_GCaMP/Calcium/GAD_hM3Dq_{mouse}/GAD_hM3Dq_{mouse}_{session}/minian'
crossreg_path = f'/media/caishuman/csstorage3/Destynie/Misc/GAD_hM3Dq_GCaMP/Calcium/GAD_hM3Dq_{mouse}/SpatialFootprints/CellRegResults'
fig_path = '/media/caishuman/csstorage3/Austin/Quality_Control/visualizations'
alpha = 0.001
nshuffles = 10
percentile = 95
bin_size = 0.2 ## in seconds
corr_metric = 'spearman'
diag_zero = True 
zthresh = 2

In [ ]:
## Load data
C = ctn.open_minian(dpath)['C']
S = ctn.open_minian(dpath)['S']
A = ctn.open_minian(dpath)['A']
max_proj = ctn.open_minian(dpath)['max_proj']

## Load cross-registration data
mappings = pd.read_csv(pjoin(crossreg_path, 'CellRegResults.csv'))

In [ ]:
## Trend test
cell_act = C.values
zdata = zscore(cell_act, axis=1)
bin_act = ctn.bin_activity(zdata, bin_size_seconds=60, func=np.min)
cell_trends = {'unit_id': [], 'trend': [], 'mk_score': [], 'pval': []}
for uid in np.arange(0, C.shape[0]):
    mkt = mk.original_test(bin_act[uid, :])
    cell_trends['unit_id'].append(uid)
    cell_trends['trend'].append(mkt.trend)
    cell_trends['mk_score'].append(np.abs(mkt.s))
    cell_trends['pval'].append(mkt.p)
trends = pd.DataFrame(cell_trends)
dropped_uids = trends[(trends['pval'] < alpha)].reset_index(drop=True)

In [ ]:
## Plot individual modeled calcium traces
fig = pf.custom_graph_template(x_title='Time (s)', y_title='Activity (a.u.)', width=700)
for neuron in dropped_uids['unit_id']:
    fig.add_trace(go.Scattergl(x=C['frame'].values, y=zdata[neuron, :], mode='lines', visible='legendonly',
                               line_color='red', name=f'Neuron {neuron}', legendgroup=f'{neuron}'))
fig.show()

In [ ]:
# Plot spatial footprint on max projection
act_bin = ctn.bin_activity(S.values, bin_size_seconds=bin_size, func=np.mean)
cor_mat = ctn.cell_cell_correlations(act_bin, corr_metric=corr_metric, diag_zero=diag_zero)

## Shuffle activity to create null distribution
seeds = np.random.randint(0, 10000, nshuffles)
shuff_mat = ctn.shuffle_cell_cell_correlations(act_bin, seeds=seeds, corr_metric=corr_metric, nshuffles=nshuffles)
sig_cor = cor_mat > np.percentile(shuff_mat, percentile, axis=2)

## Calculate single neuron cumulative co-firing strength
mat = cor_mat.copy()
mat[~sig_cor] = 0
ranked_act = np.unique(mat.flatten()[np.argsort(mat.flatten())])

## Get average activity of cells
activity = np.sum(act_bin > 0, axis=1)
ranked_activity = activity[np.argsort(activity)]

In [ ]:
## Plot an example highly connected neuron
idx = -1
if np.where(mat == ranked_act[idx])[0].shape[0] == 1:
    neurons = np.concatenate((np.where(mat == ranked_act[idx])[0], np.where(mat == ranked_act[idx])[1]))
else:
    neurons = np.where(mat == ranked_act[idx])[0] 
uids = S['unit_id'].values[neurons]
avals = A.sel(unit_id=uids).values
n1 = avals[0]
n2 = avals[1]
n1[n1 == 0] = np.nan
n2[n2 == 0] = np.nan
fig = pf.custom_graph_template(x_title='', y_title='', width=1200, height=600, rows=1, columns=2,
                               shared_x=True, shared_y=True, titles=[f'Correlation: {ranked_act[idx]}'])
# norm_vals = max_proj.values / np.max(max_proj.values)
norm_vals = max_proj.values
# norm_vals = (max_proj.values > 16).astype(int)
fig.add_trace(go.Heatmap(x=np.arange(0, max_proj.shape[1]), y=np.arange(0, max_proj.shape[0]), z=norm_vals,
                         colorscale='viridis'), row=1, col=1)
fig.add_trace(go.Heatmap(x=np.arange(0, max_proj.shape[1]), y=np.arange(0, max_proj.shape[0]), z=norm_vals,
                         colorscale='viridis'), row=1, col=2)
fig.add_trace(go.Heatmap(x=np.arange(0, max_proj.shape[1]), y=np.arange(0, max_proj.shape[0]), 
                         z=n1, colorscale='reds_r', showscale=False, opacity=1), row=1, col=2)
fig.add_trace(go.Heatmap(x=np.arange(0, max_proj.shape[1]), y=np.arange(0, max_proj.shape[0]), 
                         z=n2, colorscale='blues_r', showscale=False, opacity=1), row=1, col=2)
fig.update_layout(xaxis={'visible': False}, yaxis={'visible': False}, xaxis2_visible=False, yaxis2_visible=False)
fig.show()

## Plot cell activity binned in 200ms bins, which was used for the correlations
xaxis = np.arange(0, act_bin.shape[1]) * bin_size
fig2 = pf.custom_graph_template(x_title='Time (s)', y_title='Activity (a.u.)', width=700)
fig2.add_trace(go.Scattergl(x=xaxis, y=act_bin[neurons[0]], line_color='darkgrey', name=f'Neuron {neurons[0]}'))
fig2.add_trace(go.Scattergl(x=xaxis, y=act_bin[neurons[1]], line_color='red', name=f'Neuron {neurons[1]}', opacity=0.1))
fig2.show()

## Save images
# fig.write_image(pjoin(fig_path, f'correlated_cell_pair_idx{idx}_spatial_footprints.png'), width=1200, height=600)
# fig2.write_image(pjoin(fig_path, f'correlated_cell_pair_idx{idx}_smat_activity.png'), width=700, height=500)

In [ ]:
## Choose two neurons to plot their spatial footprints and their activity
neurons = [0, 1]
avals = A[neurons].values
n1 = avals[0]
n2 = avals[1]
n1[n1 == 0] = np.nan
n2[n2 == 0] = np.nan
fig = pf.custom_graph_template(x_title='', y_title='', width=1200, height=600, rows=1, columns=2,
                               shared_x=True, shared_y=True)
# norm_vals = max_proj.values / np.max(max_proj.values)
norm_vals = max_proj.values
fig.add_trace(go.Heatmap(x=np.arange(0, max_proj.shape[1]), y=np.arange(0, max_proj.shape[0]), z=norm_vals,
                         colorscale='viridis'), row=1, col=1)
fig.add_trace(go.Heatmap(x=np.arange(0, max_proj.shape[1]), y=np.arange(0, max_proj.shape[0]), z=norm_vals,
                         colorscale='viridis'), row=1, col=2)
fig.add_trace(go.Heatmap(x=np.arange(0, max_proj.shape[1]), y=np.arange(0, max_proj.shape[0]), 
                         z=n1, colorscale='reds_r', showscale=False, opacity=1), row=1, col=2)
fig.add_trace(go.Heatmap(x=np.arange(0, max_proj.shape[1]), y=np.arange(0, max_proj.shape[0]), 
                         z=n2, colorscale='blues_r', showscale=False, opacity=1), row=1, col=2)
fig.update_layout(xaxis={'visible': False}, yaxis={'visible': False})
fig.show()

## Plot cell activity binned in 200ms bins
xaxis = np.arange(0, act_bin.shape[1]) * bin_size
fig2 = pf.custom_graph_template(x_title='Time (s)', y_title='Activity (a.u.)', width=700)
fig2.add_trace(go.Scattergl(x=xaxis, y=act_bin[neurons[0]], line_color='darkgrey', name=f'Neuron {neurons[0]}'))
fig2.add_trace(go.Scattergl(x=xaxis, y=act_bin[neurons[1]], line_color='red', name=f'Neuron {neurons[1]}', opacity=0.1))
fig2.show()

## Save images
# fig.write_image(pjoin(fig_path, f'nearby_cell_pair_{neurons}_spatial_footprints.png'), width=1200, height=600)
# fig2.write_image(pjoin(fig_path, f'nearby_cell_pair_{neurons}_smat_activity.png'), width=700, height=500)

In [ ]:
## Plot an example of two highly active neurons
idx_one = -8
idx_two = -9
neuron_one = np.where(activity == ranked_activity[idx_one])[0][0]
neuron_two = np.where(activity == ranked_activity[idx_two])[0][0]
neurons = [neuron_one, neuron_two]
avals = A[neurons].values
n1 = avals[0] 
n2 = avals[1] 
n1[n1 == 0] = np.nan
n2[n2 == 0] = np.nan
fig = pf.custom_graph_template(x_title='', y_title='', width=1200, height=600, rows=1, columns=2,
                               shared_x=True, shared_y=True)
norm_vals = max_proj.values / np.max(max_proj.values)
fig.add_trace(go.Heatmap(x=np.arange(0, max_proj.shape[1]), y=np.arange(0, max_proj.shape[0]), z=norm_vals,
                         colorscale='gray'), row=1, col=1)
fig.add_trace(go.Heatmap(x=np.arange(0, max_proj.shape[1]), y=np.arange(0, max_proj.shape[0]), z=norm_vals,
                         colorscale='gray'), row=1, col=2)
fig.add_trace(go.Heatmap(x=np.arange(0, max_proj.shape[1]), y=np.arange(0, max_proj.shape[0]), 
                         z=n1, colorscale='reds_r', showscale=False, opacity=1), row=1, col=2)
fig.add_trace(go.Heatmap(x=np.arange(0, max_proj.shape[1]), y=np.arange(0, max_proj.shape[0]), 
                         z=n2, colorscale='blues_r', showscale=False, opacity=1), row=1, col=2)
fig.update_layout(xaxis={'visible': False}, yaxis={'visible': False})
fig.show()

## Plot the modeled calcium activity of those two cells
xaxis = np.arange(0, act_bin.shape[1]) * bin_size
fig2 = pf.custom_graph_template(x_title='Frame', y_title='Activity (a.u.)', width=700)
fig2.add_trace(go.Scattergl(x=C['frame'].values, y=C[neurons[0], :], line_color='red', name=f'Neuron {neurons[0]}'))
fig2.add_trace(go.Scattergl(x=C['frame'].values, y=C[neurons[1], :], line_color='blue', name=f'Neuron {neurons[1]}', opacity=0.1))
fig2.show()

### Look at cross-registered cells and see where they are on the max projection.

In [ ]:
## Look at cross-registered cells between the Chemotagging session and the previous saline session
shared_cells = mappings[[f'GAD_hM3Dq_{mouse}_PreSession2', f'GAD_hM3Dq_{mouse}_{session}']]
registered = shared_cells[shared_cells > -9999].dropna().reset_index(drop=True).astype(int)

## Choose x random neurons
num_cells = 75
random = np.random.randint(0, registered.shape[0], num_cells)
subset = registered.iloc[random]

## Get spatial footprint values
avals = A.sel(unit_id=subset[f'GAD_hM3Dq_{mouse}_{session}'].to_numpy()).values
footprints = np.mean(avals, axis=0)
footprints[footprints == 0] = np.nan ## set zero values to nan for plotting
thresh = ctn.calculate_otsu_thresh(max_proj.values)

In [ ]:
# norm_vals = max_proj.values / np.max(max_proj.values) ## normalize from zero to one
norm_vals = (max_proj.values > thresh).astype(int)
fig = pf.custom_graph_template(x_title='', y_title='', width=1200, height=600, rows=1, columns=2,
                               shared_x=True, shared_y=True)
fig.add_trace(go.Heatmap(x=np.arange(0, max_proj.shape[1]), y=np.arange(0, max_proj.shape[0]), z=norm_vals,
                         colorscale='gray'), row=1, col=1)
fig.add_trace(go.Heatmap(x=np.arange(0, max_proj.shape[1]), y=np.arange(0, max_proj.shape[0]), z=norm_vals,
                         colorscale='gray'), row=1, col=2)
fig.add_trace(go.Heatmap(x=np.arange(0, max_proj.shape[1]), y=np.arange(0, max_proj.shape[0]), 
                         z=footprints, colorscale='reds_r', showscale=False, opacity=1), row=1, col=2)
fig.show()

In [ ]:
## Get the spatial footprints of the overlapping units
spatial_footprints = A[registered[f'GAD_hM3Dq_{mouse}_{session}'].to_numpy()].values ## select cross-registered cells
proj_vals = max_proj.values
output_dict = {'unit_id': [], 'weighted_sum': []}
for uid in np.arange(0, spatial_footprints.shape[0]):
    row, col = np.where(spatial_footprints[uid] > 0)[0], np.where(spatial_footprints[uid] > 0)[1]
    output_dict['unit_id'].append(uid)
    output_dict['weighted_sum'].append(np.average(proj_vals[row, col], weights=spatial_footprints[uid, row, col]))
output_df = pd.DataFrame(output_dict)

## Plot histogram of all cross-registered cells weighted average pixel value
lq = np.percentile(output_df['weighted_sum'], 25)
H, bins = np.histogram(output_df['weighted_sum'], bins=15)
H_lq, bins_lq = np.histogram(output_df['weighted_sum'][output_df['weighted_sum'] < lq], bins=15)
fig = pf.custom_graph_template(x_title='Pixel Weighted Sum', y_title='Count')
fig.add_trace(go.Bar(x=bins[:-1], y=H, marker_color='darkgrey', marker_line_width=2, marker_line_color='black'))
fig.add_vline(x=lq, line_width=3, line_dash='dash', opacity=1)
fig.show()

In [ ]:
## Plot histogram of all cross-registered cells weighted average pixel value
## Histogram settings
num_bins = 15
histogram_bins = np.linspace(0, np.max(output_df['weighted_sum']), num_bins)
lq = np.percentile(output_df['weighted_sum'], 25) ## gets the value for the 25th percentile
uq = np.percentile(output_df['weighted_sum'], 75)

H, bins = np.histogram(output_df['weighted_sum'], bins=histogram_bins)
H_lq, bins_lq = np.histogram(output_df['weighted_sum'][output_df['weighted_sum'] < lq], bins=histogram_bins)
H_uq, bins_uq = np.histogram(output_df['weighted_sum'][output_df['weighted_sum'] > uq], bins=histogram_bins)

lq_cells = output_df['unit_id'][output_df['weighted_sum'] < lq].to_numpy()
uq_cells = output_df['unit_id'][output_df['weighted_sum'] > uq].to_numpy()

fig = pf.custom_graph_template(x_title='Pixel Weighted Sum', y_title='Count')
fig.add_trace(go.Bar(x=bins[:-1], y=H, marker_color='darkgrey', marker_line_width=2, marker_line_color='black', showlegend=False))
fig.add_trace(go.Bar(x=bins_lq[:-1], y=H_lq, marker_color='#daa635', marker_line_width=2, marker_line_color='black', showlegend=False))
fig.add_trace(go.Bar(x=bins_lq[:-1], y=H_uq, marker_color='#2fcce4', marker_line_width=2, marker_line_color='black', showlegend=False))
fig.update_layout(barmode='overlay')
fig.show()

In [ ]:
## Plot spatial footprints of the neurons below the weighted sum of pixel values below the 25th percentile
norm_vals = max_proj.values / np.max(max_proj.values) ## normalize from zero to one
fig = pf.custom_graph_template(x_title='', y_title='', width=1200, height=600, rows=1, columns=2,
                               shared_x=True, shared_y=True)
fig.add_trace(go.Heatmap(x=np.arange(0, max_proj.shape[1]), y=np.arange(0, max_proj.shape[0]), z=norm_vals,
                         colorscale='gray'), row=1, col=1)
fig.add_trace(go.Heatmap(x=np.arange(0, max_proj.shape[1]), y=np.arange(0, max_proj.shape[0]), z=norm_vals,
                         colorscale='gray'), row=1, col=2)

bt_spatial = spatial_footprints[lq_cells]
avg_spatial = np.mean(bt_spatial, axis=0)
avg_spatial[avg_spatial == 0] = np.nan
fig.add_trace(go.Heatmap(x=np.arange(0, max_proj.shape[1]), y=np.arange(0, max_proj.shape[0]), 
                         z=avg_spatial, colorscale='reds_r', showscale=False, opacity=1), row=1, col=2)
print(f'Percentage of cells in the lower quartile: {(bt_spatial.shape[0] / registered.shape[0]) * 100}')
fig.show()

In [ ]:
## Plot spatial footprints of the neurons below the weighted sum of pixel values below the 25th percentile
norm_vals = max_proj.values / np.max(max_proj.values) ## normalize from zero to one
fig = pf.custom_graph_template(x_title='', y_title='', width=1200, height=600, rows=1, columns=2,
                               shared_x=True, shared_y=True)
fig.add_trace(go.Heatmap(x=np.arange(0, max_proj.shape[1]), y=np.arange(0, max_proj.shape[0]), z=norm_vals,
                         colorscale='gray'), row=1, col=1)
fig.add_trace(go.Heatmap(x=np.arange(0, max_proj.shape[1]), y=np.arange(0, max_proj.shape[0]), z=norm_vals,
                         colorscale='gray'), row=1, col=2)

bt_spatial = spatial_footprints[uq_cells]
avg_spatial = np.mean(bt_spatial, axis=0)
avg_spatial[avg_spatial == 0] = np.nan
fig.add_trace(go.Heatmap(x=np.arange(0, max_proj.shape[1]), y=np.arange(0, max_proj.shape[0]), 
                         z=avg_spatial, colorscale='blues_r', showscale=False, opacity=1), row=1, col=2)
print(f'Percentage of cells in the upper quartile: {(bt_spatial.shape[0] / registered.shape[0]) * 100}')
fig.show()

### Look at whether cells that are likely not cells participate in bursts.

In [ ]:
## Zscore each cell, then take the average of the zscore, then zscore again
## Bursts_og is the original burst definition where any frame above zthresh of 2 is considered a separate burst
zdata = zscore(S.values, axis=1)
pop_act = zscore(np.mean(zdata, axis=0))
bursts_og = pop_act > zthresh

In [ ]:
## Plot bursts below linearized position and include raster of cell firing
xaxis = S['frame'].values
fig = pf.custom_graph_template(x_title='Frame', y_title='Population Activity', font_size=22, titles=[f'{mouse}' + ' ' + f'{session}'],
                               height=500, width=1000)
## Population bursts
fig.add_trace(go.Scattergl(x=xaxis, y=pop_act, mode='lines', line_color='black', showlegend=False))
fig.add_trace(go.Scattergl(x=xaxis[bursts_og], y=pop_act[bursts_og], mode='markers', marker_color='red', showlegend=False))
fig.add_hline(y=zthresh, line_width=1, line_color='red', line_dash='dash', opacity=1)
fig.show()

In [ ]:
sdata = S.values 
lq_s = sdata[lq_cells, :] > 0
uq_s = sdata[uq_cells, :] > 0

part_bursts_lq = (np.sum(lq_s[:, bursts_og], axis=1) / bursts_og.shape[0]) * 100
part_bursts_uq = (np.sum(uq_s[:, bursts_og], axis=1) / bursts_og.shape[0]) * 100

bursts_lq = zscore(np.mean(zscore(lq_s, axis=1), axis=0)) > zthresh
bursts_uq = zscore(np.mean(zscore(uq_s, axis=1), axis=0)) > zthresh

In [ ]:
fig = pf.custom_graph_template(x_title='', y_title='Participation (%)', titles=['Participation in OG Burst'])
fig.add_trace(go.Bar(x=['Lower Quartile'], y=[np.mean(part_bursts_lq)], marker_color='#daa635', showlegend=False,
                     error_y=dict(type='data', array=[np.mean(np.sqrt(np.std(part_bursts_lq, ddof=1) / part_bursts_lq.shape[0]))])))
fig.add_trace(go.Bar(x=['Upper Quartile'], y=[np.mean(part_bursts_uq)], marker_color='#2fcce4', showlegend=False,
                     error_y=dict(type='data', array=[np.mean(np.sqrt(np.std(part_bursts_uq, ddof=1) / part_bursts_uq.shape[0]))])))
fig.show()

In [ ]:
from scipy.stats import spearmanr
print(f'Correlation lower quartile bursts: {spearmanr(bursts_lq, bursts_og).statistic}')
print(f'Correlation upper quartile bursts: {spearmanr(bursts_uq, bursts_og).statistic}')